# Wear predition 
## <font color="#dd0000">Group A</font> --  unary descriptor
### generating and screening **unary** descriptor by *LASSO+L0*

In [7]:
import os
import pandas as pd
import numpy as np
from itertools import combinations
from time import time
import matplotlib.pyplot as plt 
import scipy.stats as ss
import warnings
from collections import Counter
from sklearn.kernel_ridge import KernelRidge
from IPython.display import HTML
from jupyter_jsmol import JsmolView
import itertools
import functools
import math
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


import combine_features as cf
warnings.filterwarnings('ignore')
pd.set_option('mode.use_inf_as_na', True)

In [8]:
df = pd.read_csv("groupA.csv")
df.head()
df.head()

,Sample No.,Hv (kg/mm2),UTS (MPa),YS (MPa),UE (%),ETF (104 Pa),K (MPa),n,WL(g)
0,A1,178,515,311,16.40,15292,1034,0.26,0.7889
1,A2,311,839,456,8.25,9554,2080,0.28,0.6847
2,A3,367,949,486,5.95,6137,3386,0.36,0.6257
3,A4,399,1147,612,5.65,7747,3929,0.34,0.5757
4,A5,426,1226,683,5.20,10370,3901,0.33,0.6072


In [9]:
df.rename(columns={'ETF\n(104 Pa)':'ETF(MPa)'},inplace=True)
df['ETF(MPa)']=df['ETF(MPa)']/100 # 把etf的单位变为mpa
df.columns=['Sample No.','Hv','UTS','YS','UE','ETF','K','n','WL']
df['UE']=df['UE']/100 # 把ue变成小数
df['Hv']=df['Hv']*9.8  #change the unit of Hv into MPa

df.head()

,Sample No.,Hv,UTS,YS,UE,ETF,K,n,WL
0,A1,1744.4,515,311,0.1640,152.92,1034,0.26,0.7889
1,A2,3047.8,839,456,0.0825,95.54,2080,0.28,0.6847
2,A3,3596.6,949,486,0.0595,61.37,3386,0.36,0.6257
3,A4,3910.2,1147,612,0.0565,77.47,3929,0.34,0.5757
4,A5,4174.8,1226,683,0.0520,103.70,3901,0.33,0.6072


In [10]:
df.drop(['Sample No.'],axis=1,inplace=True)

df.head()

,Hv,UTS,YS,UE,ETF,K,n,WL
0,1744.4,515,311,0.1640,152.92,1034,0.26,0.7889
1,3047.8,839,456,0.0825,95.54,2080,0.28,0.6847
2,3596.6,949,486,0.0595,61.37,3386,0.36,0.6257
3,3910.2,1147,612,0.0565,77.47,3929,0.34,0.5757
4,4174.8,1226,683,0.0520,103.70,3901,0.33,0.6072


In [11]:
# define the calculation method
_my_sum = lambda x:np.sum(x)
_my_diff= lambda x:np.diff(x)
_my_abs_diff= lambda x:np.abs(_my_diff(x))

_my_power_2=lambda x: np.power(x,2)

def _my_div(x):
    return x[0]/x[1]
_my_mul= lambda x:np.prod(x)

In [12]:
def combine_features(df=None, allowed_operations=None, is_print=True):
### generate combinatons of features by given a dataframe and a list of allowed operations.
    if is_print:
        if allowed_operations:
            print('Selected operations:\\n {0}'.format(allowed_operations))
        else:
            print('No allowed operations selected.')
        columns_ = df.columns.tolist()
    dict_features={
        ### groupby original data according to the unit
        'UTS':'a1',
        'YS':'a1',
        'ETF':'a1',
        'K':'a1',
        'Hv':'a1', 
        
        'UE':'a2',
        'n':'a2',
    
        }
    df_a1 = df[[col for col in columns_ if dict_features.get(col.split('(', 1)[0])=='a1']].astype('float32')
    df_a2 = df[[col for col in columns_ if dict_features.get(col.split('(', 1)[0])=='a2']].astype('float32')
  
    col_a1 = df_a1.columns.tolist()
    col_a2 = df_a2.columns.tolist()

    ## this list will at the end all the dataframes created\n',
    df_list = []
    
    df_b1_list = []
    df_b2_list = []
    df_c1_list = []
    df_c2_list = []
    df_d_list = []
    df_e_list = []
    
    df_x1_list = []
    df_x2_list = []

    # create b1: absolute differences and sums of a1
    for subset in itertools.combinations(col_a1,2):
        if '+' in allowed_operations:
            cols = ['('+subset[0]+'+'+subset[1]+')']
            data = df_a1[list(subset)].apply(_my_sum,axis=1)
            df_b1_list.append(pd.DataFrame(data,columns=cols))
            
        if '-' in allowed_operations:
            cols = ['('+subset[1]+'-'+subset[0]+')']
            data = df_a1[list(subset)].apply(_my_diff,axis=1)
            df_b1_list.append(pd.DataFrame(data,columns=cols))
            
        if '|-|' in allowed_operations:
            cols = ['|'+subset[1]+'-'+subset[0]+'|']
            data = df_a1[list(subset)].apply(_my_abs_diff,axis=1)
            df_b1_list.append(pd.DataFrame(data,columns=cols))
            
                   
    #create b2: abs diff and sums of a2:
    for subset in itertools.combinations(col_a2,2):
        if '+' in allowed_operations:
            cols = ['('+subset[0]+'+'+subset[1]+')']
            data = df_a2[list(subset)].apply(_my_sum,axis=1)
            df_b2_list.append(pd.DataFrame(data,columns=cols))

            
        if '-' in allowed_operations:
            cols = ['('+subset[1]+'-'+subset[0]+')']
            data = df_a2[list(subset)].apply(_my_diff,axis=1)
            df_b2_list.append(pd.DataFrame(data,columns=cols))

            
        if '|-|' in allowed_operations:
            cols = ['|'+subset[1]+'-'+subset[0]+'|']
            data = df_a2[list(subset)].apply(_my_abs_diff,axis=1)
            df_b2_list.append(pd.DataFrame(data,columns=cols))
            
    # create c1:^2 of a1:
    for subset in itertools.combinations(col_a1,1):

        if '^2' in allowed_operations:
            cols = ['('+subset[0]+'^2'+')']       
            f = df_a1[list(subset)].apply(_my_power_2,axis=1)
            data=f.values
            df_c1_list.append(pd.DataFrame(data,columns=cols))
        if '^3' in allowed_operations:
            cols = ['('+subset[0]+'^3'+')']       
            f = df_a1[list(subset)].apply(_my_power_3,axis=1)
            data=f.values
            df_c1_list.append(pd.DataFrame(data,columns=cols))
            
    # create c2:^2 of a2:
    for subset in itertools.combinations(col_a2,1):

        if '^2' in allowed_operations:
            cols = ['('+subset[0]+'^2'+')']       
            f = df_a2[list(subset)].apply(_my_power_2,axis=1)
            data=f.values
            df_c2_list.append(pd.DataFrame(data,columns=cols))
        if '^3' in allowed_operations:
            cols = ['('+subset[0]+'^3'+')']       
            f = df_a2[list(subset)].apply(_my_power_3,axis=1)
            data=f.values
            df_c2_list.append(pd.DataFrame(data,columns=cols))
            
            

            
        
    if not df_a1.empty: 
        df_x1_list.append(df_a1)
        df_list.append(df_a1)
        
    if not df_a2.empty: 
        df_x1_list.append(df_a2)
        df_list.append(df_a2)

        
    if df_b1_list: 
        df_b1 = pd.concat(df_b1_list, axis=1)
        col_b1 = df_b1.columns.tolist()
        #df_x1_list.append(df_b1)
        df_list.append(df_b1)

    if df_b2_list: 
        df_b2 = pd.concat(df_b2_list, axis=1)
        col_b2 = df_b2.columns.tolist()
        #df_x1_list.append(df_b2)
        df_list.append(df_b2)
        
    if df_c1_list: 
        df_c1 = pd.concat(df_c1_list, axis=1)
        col_c1 = df_c1.columns.tolist()
        df_x1_list.append(df_c1)
        df_list.append(df_c1)
    if df_c2_list: 
        df_c2 = pd.concat(df_c2_list, axis=1)
        col_c2 = df_c2.columns.tolist()
        df_x1_list.append(df_c2)
        df_list.append(df_c2)
        
    if df_x1_list:
        df_x1 = pd.concat(df_x1_list, axis=1)
        col_x1 = df_x1.columns.tolist()
        
    if df_x2_list:
        df_x2 = pd.concat(df_x2_list, axis=1)
        col_x2 = df_x2.columns.tolist()
        
    #create d: mul within X1:
    for subset in itertools.combinations(col_x1,2):
                if '*' in allowed_operations:
                    #print(subset[0],subset[1])
                    cols = [subset[0]+'*'+subset[1]] 
                    #now the operation is between two dataframes\n',
                    data = df_x1[list(subset)].apply(_my_mul, axis=1)  
                    df_d_list.append(pd.DataFrame(data, columns=cols)) 

                if '/' in allowed_operations:
                    cols = [subset[0]+'/'+subset[1]] 
                    #now the operation is between two dataframes\n',
                    data = df_x1[list(subset)].apply(_my_div, axis=1) 
                    df_d_list.append(pd.DataFrame(data, columns=cols)) 
                    #reverse
                    cols = [subset[1]+'/'+subset[0]] 
                    data = df_x1[list(subset)].apply(_my_div, axis=1) 
                    df_d_list.append(pd.DataFrame(data, columns=cols)) 
                    
    if df_d_list:
        df_d=pd.concat(df_d_list,axis=1)
        col_d=df_d.columns.tolist()
        df_list.append(df_d)
    if df_e_list:
        df_e=pd.concat(df_e_list,axis=1)
        col_e=df_e.columns.tolist()
        df_list.append(df_e)

    if df_list:
        df_combined_features = pd.concat(df_list, axis=1)
        print(len(df_b1_list),len(df_b2_list),len(df_d_list),len(df_e_list))
    elif is_print:
        print('No features selected. Please select at least two primary features.')


    if is_print:
        print('Number of total features generated: {0}'.format(df_combined_features.shape[1]))

    return df_combined_features 
            

In [13]:
def get_data(selected_feature_list, allowed_operations):
    # extract WEARLOSS and selected features from df_data 
    P = df['WL'].values
    df_features = df[selected_feature_list]
    
    # derive new features using allowed_operations
    df_combined = combine_features(df=df_features, allowed_operations=allowed_operations)
    return P, df_combined

In [14]:
#import Data
selected_feature_list = ['UE','Hv','UTS','YS','n','ETF','K']
## determine the allowed operators
allowed_operations = []



P, df_D = get_data(selected_feature_list, allowed_operations)
df_D.replace([np.inf, -np.inf], np.nan,inplace=True)
df_D =df_D.dropna(axis=1)
df_D = df_D.loc[:, (df_D != 0).any(axis=0)]



D = df_D.values
D = D.astype(np.float64)
features_list = df_D.columns.tolist()




No allowed operations selected.
0 0 0 0
Number of total features generated: 7


In [15]:
pd.set_option('display.width', 10)  # 设置字符显示宽度
pd.set_option('display.max_rows', None)  # 设置显示最大行
print(df_D.iloc[0])
df_D.iloc[0].isnull().values.any()

Hv     1744.400024
UTS     515.000000
YS      311.000000
ETF     152.919998
K      1034.000000
UE        0.164000
n         0.260000
Name: 0, dtype: float32


False

In [16]:
# standardize data
D_standardized = ss.zscore(D)
print(D_standardized.shape)
print(type(D_standardized))
np.isnan(D_standardized).any()
D_standardized=D_standardized[:, ~np.isnan(D_standardized).any(axis=0)]

(20, 7)
<class 'numpy.ndarray'>


In [17]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, D_standardized, P, 
                                   scoring="neg_mean_squared_error", cv = 5))
    return(rmse)



In [18]:
##cv调参2
# 根据上图建立Lasso进行alpha选择的范围
alpha_range = np.linspace(0.1,0.001,70)

# # LassoCV
lasso_ = LassoCV(alphas=alpha_range,cv=5).fit(D_standardized,P)

# # 查看最佳正则化系数
best_alpha = lasso_.alpha_ 
print(best_alpha)




0.0024347826086956598


In [19]:
def rmse():
    
    P_predict = lasso.predict(D_standardized)
    #print(P_predict)
    RMSE_LASSO = np.linalg.norm(P-P_predict) / np.sqrt(20.)
    return RMSE_LASSO




In [20]:
from sklearn.metrics import r2_score
def r_2():
    P_predict = lasso.predict(D_standardized)
    return r2_score(P_predict,P)



In [21]:
#以线性关系改变lambda的值，观察非零系数的个数
from sklearn.linear_model import Lasso
alpha = np.linspace(0.1,0.01,90)
m=0
lams=[]
rmse_s=[]
#table = PrettyTable(['NO.','alpha', 'num_nonzero','selected features','rmse','coef','r^2','intercept'])


for i in range(90):
    global m
    lasso=Lasso(alpha=alpha[i])
    lasso.fit(D_standardized,P)
    coef=lasso.coef_
    a=np.sum(coef != 0)
    
    if a!=m:
        m=a
        
#         print(m)
#         print(alpha[i])

        
    # get strings of selected features
        selected_indices = lasso.coef_.nonzero()[0]
        selected_features = [features_list[i] for i in selected_indices]
        #print(lasso.predict(D_standardized))
        #print(pd.DataFrame({'alpha':"%.3f" %alpha[i],'num_nonzero':m,'features_list':selected_features,'rmse':rmse_cv(),'coef':coef[coef.nonzero()].round(3)}))
        #print('alpha',"%.3f" %alpha[i],'num_nonzero',m,selected_features,'rmse',rmse_cv(),'coef',coef[coef.nonzero()].round(5))
        #pd.Series(index = ['Intercept'] + D_standardized.columns.tolist(),data = [lasso.intercept_] + lasso.coef_.tolist())
        #print(lasso.intercept_)
        if a<=10:
            #table.add_row([i,alpha[i].round(2),m,selected_features,rmse().round(2),coef[coef.nonzero()].round(2),lasso.intercept_.round(2),r_2()])
            print('alpha',"%.5f" %alpha[i],'num_nonzero',m,selected_features,'rmse',rmse().round(4),'coef',coef[coef.nonzero()].round(5),'截距:',lasso.intercept_.round(2),'\n','r_squared:',r_2().round(5))
            
            rmse_s.append(rmse)
            lams.append(alpha[i])



# print("lambda: %.3f\t dimension of descriptor: %s\t RMSE_LASSO: %.3f\t RMSE_LS: %.3f" 
#       %(lam, len(selected_features), RMSE_LASSO, RMSE_LS))
# print(pd.DataFrame({'features':np.array(selected_features), 'abs(nonzero_coefs_LASSO)': np.abs(coef[coef.nonzero()])}))
# plt.show()  

alpha 0.07371 num_nonzero 1 ['K'] rmse 0.0805 coef [-2.e-05] 截距: 0.66 
 r_squared: -17360830.27068
alpha 0.04742 num_nonzero 2 ['Hv', 'K'] rmse 0.0573 coef [-0.00047 -0.02589] 截距: 0.66 
 r_squared: -3.74358
alpha 0.03225 num_nonzero 3 ['Hv', 'K', 'UE'] rmse 0.0448 coef [-0.00818 -0.03399  0.00023] 截距: 0.66 
 r_squared: -0.15687
alpha 0.01910 num_nonzero 2 ['K', 'UE'] rmse 0.0346 coef [-0.04611  0.01208] 截距: 0.66 
 r_squared: 0.60886


In [22]:
model=Lasso(best_alpha).fit(D_standardized,P)
coef=model.coef_
a=np.sum(coef != 0)
selected_indices = model.coef_.nonzero()[0]
selected_features = [features_list[i] for i in selected_indices]
print('num_nonzero',a,selected_features,'rmse',rmse(),'coef',coef[coef.nonzero()].round(8),'截距:',model.intercept_)


num_nonzero 3 ['YS', 'ETF', 'K'] rmse 0.029754571984185355 coef [-0.02376908  0.01845572 -0.05307079] 截距: 0.66381


In [23]:
# candidate descriptors screened by LASSO
D_screened=df_D[selected_features]
D_= D_screened.values
D_= D_.astype(np.float64)
D_screened.info()
D_screened_standardized = ss.zscore(D_)
print(len(D_screened_standardized[0]))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
YS     20 non-null float32
ETF    20 non-null float32
K      20 non-null float32
dtypes: float32(3)
memory usage: 320.0 bytes
3


In [24]:
def L0(P, D, dimension):
    n_rows, n_columns = D.shape
    D = np.column_stack((D, np.ones(n_rows)))
    SE_min = np.inner(P ,P)
    coef_min, permu_min = None, None
    for permu in combinations(range(n_columns), dimension):
        D_ls = D[:, permu + (-1,)]
        coef, SE, __1, __2 = np.linalg.lstsq(D_ls, P, rcond=-1)
        try:
            if SE[0] < SE_min: 
                SE_min = SE[0]
                coef_min, permu_min = coef, permu
        except:
            pass
    RMSE = np.sqrt(SE_min/n_rows)
    return RMSE, coef_min, permu_min

In [25]:
def L0_predict(coefficients,X,selected_indices):
    X = np.array(X)
    p=len(X)
    y_pred=[]
    n=len(selected_indices)
    X_selected=np.zeros(shape=((n,p)))
    w=coefficients
    m=len(coefficients)
    print('len(coefs)=',m)

    z=0
    for j in selected_indices:
        X_selected[z,:]=X[:,j]
        z=z+1

    for i in range(0,p):

        ele=np.dot(w[0:m-1],X_selected[:,i])+w[m-1]
        y_pred.append(ele.round(4))
    
    
    return y_pred

In [32]:
# descriptor selected by L0
print("     RMSE   best desriptor")
for dim in range(1,2):
    RMSE, coefficients, selected_indices_l0 = L0(P,D_screened_standardized,dim)
    Y_pre=L0_predict(coefficients,D_screened_standardized,selected_indices_l0)
    print(Y_pre)
    y_pre= pd.DataFrame(Y_pre, columns=['Predict value'])
    y_pre.to_excel('/Users/zy/Desktop/lasso+l0_abrasion_FV/WL_pre_A1.xlsx')
    score=r2_score(P,Y_pre)
    descriptor=[selected_features[i] for i in selected_indices_l0]
    print('%2sD: %.5f' % (dim, RMSE), [selected_features[i] for i in selected_indices_l0],coefficients,'score',score)
    df_A_1=df[descriptor]
    df_A_1.to_excel('/Users/zy/Desktop/lasso+l0_abrasion_FV/dfA1.xlsx')

     RMSE   best desriptor
len(coefs)= 2
[0.7705, 0.7095, 0.6334, 0.6017, 0.6033, 0.7769, 0.6928, 0.6315, 0.5907, 0.5957, 0.77, 0.6877, 0.6749, 0.6506, 0.6609, 0.4793, 0.608, 0.665, 0.7041, 0.7696]
 1D: 0.03226 ['K'] [-0.07372718  0.66381   ] score 0.8393086416062512
